<a href="https://colab.research.google.com/github/GaurangRawat/NLP/blob/main/NLP_6_Fake_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [6]:
##Drop Nan Values
df=df.dropna()

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df.shape

(18285, 5)

In [9]:
### Get the Independent Features
X=df.drop('label',axis=1)

In [10]:
### Get the Dependent Features
y=df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
y.shape

(18285,)

In [13]:
import tensorflow as tf

In [15]:
tf.__version__

'2.17.1'

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing .text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [17]:
## Vocabulary Size
voc_size=5000

## Onehot Representation
---

In [18]:
messages=X.copy()

In [19]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [25]:
messages.reset_index(inplace=True)

In [26]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [27]:
import nltk
import re
from nltk.corpus import stopwords

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
## Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]',' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [30]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [31]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [32]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[3945, 2999, 2416, 3270, 4934, 1863, 3544, 2350, 4913, 217],
 [4467, 418, 4340, 1638, 3022, 3582, 4125],
 [1727, 2193, 4854, 2631],
 [4698, 1771, 4885, 235, 1850, 2834],
 [400, 3022, 375, 3503, 24, 491, 3022, 3048, 3786, 4041],
 [3646,
  939,
  2919,
  3021,
  1669,
  3949,
  1676,
  4048,
  4022,
  4184,
  3004,
  817,
  1847,
  1295,
  4125],
 [217, 782, 919, 3375, 615, 4411, 1453, 2236, 3799, 4336, 2016],
 [934, 2951, 4177, 2490, 3953, 1877, 3949, 3706, 3799, 4336, 2016],
 [3343, 3797, 4491, 764, 3767, 3764, 2952, 458, 3949, 222],
 [690, 3090, 1259, 234, 4488, 2468, 3082, 2597],
 [574, 2364, 1380, 2191, 2937, 1840, 3163, 512, 3012, 495, 3720],
 [235, 3859, 4934, 3764, 3949, 3953],
 [56, 4182, 980, 3037, 2288, 2312, 1373, 2753, 637],
 [2326, 4727, 2370, 1777, 4648, 2847, 3789, 3799, 4336, 2016],
 [1122, 2616, 4826, 925, 4419, 3799, 4336, 2016],
 [141, 4595, 4210, 3255, 4150, 2532, 751, 915, 2845, 325],
 [199, 4749, 418],
 [3952, 2466, 3777, 4391, 3949, 899, 259, 4125],
 [2158, 380, 

In [33]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [35]:
onehot_repr[1]

[4467, 418, 4340, 1638, 3022, 3582, 4125]

##Embedding Representation
---

In [36]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[3945 2999 2416 ...    0    0    0]
 [4467  418 4340 ...    0    0    0]
 [1727 2193 4854 ...    0    0    0]
 ...
 [3012  649 4536 ...    0    0    0]
 [3132 3953 4668 ...    0    0    0]
 [ 352 4084 4645 ...    0    0    0]]


In [37]:
embedded_docs[1]

array([4467,  418, 4340, 1638, 3022, 3582, 4125,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [38]:
embedded_docs[0]

array([3945, 2999, 2416, 3270, 4934, 1863, 3544, 2350, 4913,  217,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [42]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.build(input_shape=(None, sent_length))
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 100)                 │          56,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

None


In [43]:
len(embedded_docs),y.shape

(18285, (18285,))

In [44]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [45]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training
---

In [47]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.7789 - loss: 0.4255 - val_accuracy: 0.9218 - val_loss: 0.1869
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.9517 - loss: 0.1293 - val_accuracy: 0.9181 - val_loss: 0.1919
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 44ms/step - accuracy: 0.9696 - loss: 0.0897 - val_accuracy: 0.9180 - val_loss: 0.2130
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.9818 - loss: 0.0587 - val_accuracy: 0.9145 - val_loss: 0.2949
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.9857 - loss: 0.0469 - val_accuracy: 0.9095 - val_loss: 0.2967
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 13s 53ms/step - accuracy: 0.9900 - loss: 0.0347 - val_accuracy: 0.9079 - val_loss: 0.3138
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.9906 - loss: 0.0313 - val_accuracy: 0.9077 - val_loss: 0.3812
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.9913 - loss: 0.0269 - val_

## Performance Metrics and Accuracy
---

In [48]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step


In [49]:
y_pred=np.where(y_pred > 0.6,1,0) ## AUC ROC Curve

In [50]:
from sklearn.metrics import confusion_matrix

In [51]:
confusion_matrix(y_test,y_pred)

array([[3105,  314],
       [ 248, 2368]])

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9068765534382767

In [53]:
from sklearn.metrics import classification_report

In [54]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



## Adding Dropout
---

In [ ]:
"""
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])



"""